# 1. Import and Install Dependencies

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1  opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
#         draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [ ]:
len(results.left_hand_landmarks.landmark)

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [ ]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [ ]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() 
    if results.face_landmarks 
    else np.zeros(1404)


In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
result_test = extract_keypoints(results)

In [ ]:
result_test

In [ ]:
np.save('0', result_test)

In [8]:
np.load('0.npy')

FileNotFoundError: [Errno 2] No such file or directory: '0.npy'

# 4. Setup Folders for Collection

In [9]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data2') 

# Actions that we try to detect
actions = np.array(['Ola', 'Obrigado','Genial'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [10]:
# for action in actions: 
#     dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
#     for sequence in range(1,no_sequences+1):
#         try: 
#             os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
#         except:
#             pass


for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [11]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [12]:
import tensorflow
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

2022-08-29 11:03:33.305867: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: não é possível abrir arquivo compartilhado: Arquivo ou diretório inexistente; LD_LIBRARY_PATH: /home/lazaro/Documentos/anaconda3/envs/VC_env/lib/python3.7/site-packages/cv2/../../lib64:
2022-08-29 11:03:33.305889: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
label_map = {label:num for num, label in enumerate(actions)}

In [14]:
label_map

{'Ola': 0, 'Obrigado': 1, 'Genial': 2}

In [15]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [16]:
np.array(sequences).shape

(90, 30, 1662)

In [18]:
X = np.array(sequences)

In [19]:
X.shape

(90, 30, 1662)

In [20]:
y = to_categorical(labels).astype(int)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

In [44]:
y_test.shape

(9, 3)

# 7. Build and Train LSTM Neural Network

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [46]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

2022-08-29 11:19:21.049331: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-08-29 11:19:21.049355: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-08-29 11:19:21.049793: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-08-29 11:19:21.049862: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1487] CUPTI activity buffer flushed


In [47]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [48]:
type(actions.shape)

tuple

In [27]:
# opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [49]:
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [50]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
2/3 [===================>..........] - ETA: 0s - loss: 2.0835 - categorical_accuracy: 0.2578

2022-08-29 11:19:33.789125: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-08-29 11:19:33.789156: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-08-29 11:19:33.969423: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.


3/3 [==============================] - 2s 281ms/step - loss: 2.7888 - categorical_accuracy: 0.2524
Epoch 2/2000
3/3 [==============================] - 0s 39ms/step - loss: 9.5237 - categorical_accuracy: 0.3314


2022-08-29 11:19:33.969878: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1487] CUPTI activity buffer flushed
2022-08-29 11:19:34.119878: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:228]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-08-29 11:19:34.140735: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-08-29 11:19:34.186915: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: Logs/train/plugins/profile/2022_08_29_11_19_34
2022-08-29 11:19:34.226334: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to Logs/train/plugins/profile/2022_08_29_11_19_34/CIS-Ubuntu.trace.json.gz
2022-08-29 11:19:34.249349: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: Logs/train/plugins/profile/2022_08_29_11_19_34
2022-08-29 11:19:34.249425: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumpe

Epoch 3/2000
3/3 [==============================] - 0s 38ms/step - loss: 10.9081 - categorical_accuracy: 0.3978
Epoch 4/2000
3/3 [==============================] - 0s 37ms/step - loss: 10.4646 - categorical_accuracy: 0.4556
Epoch 5/2000
3/3 [==============================] - 0s 38ms/step - loss: 19.7186 - categorical_accuracy: 0.3750
Epoch 6/2000
3/3 [==============================] - 0s 39ms/step - loss: 26.3817 - categorical_accuracy: 0.3375
Epoch 7/2000
3/3 [==============================] - 0s 40ms/step - loss: 19.2101 - categorical_accuracy: 0.4358
Epoch 8/2000
3/3 [==============================] - 0s 49ms/step - loss: 15.5153 - categorical_accuracy: 0.3375
Epoch 9/2000
3/3 [==============================] - 0s 39ms/step - loss: 22.3666 - categorical_accuracy: 0.2575
Epoch 10/2000
3/3 [==============================] - 0s 38ms/step - loss: 12.5970 - categorical_accuracy: 0.4007
Epoch 11/2000
3/3 [==============================] - 0s 40ms/step - loss: 24.7948 - categorical_accurac

...

3/3 [==============================] - 0s 37ms/step - loss: 0.0366 - categorical_accuracy: 0.9938
Epoch 1661/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.0503 - categorical_accuracy: 0.9899
Epoch 1662/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.0385 - categorical_accuracy: 0.9938
Epoch 1663/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.0502 - categorical_accuracy: 0.9899
Epoch 1664/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.0494 - categorical_accuracy: 0.9899
Epoch 1665/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.0727 - categorical_accuracy: 0.9821
Epoch 1666/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.0735 - categorical_accuracy: 0.9821
Epoch 1667/2000
3/3 [==============================] - 0s 47ms/step - loss: 0.0490 - categorical_accuracy: 0.9899
Epoch 1668/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.0490 - categorical_accurac

3/3 [==============================] - 0s 36ms/step - loss: 0.0475 - categorical_accuracy: 0.9899
Epoch 1733/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.0476 - categorical_accuracy: 0.9899
Epoch 1734/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0723 - categorical_accuracy: 0.9821
Epoch 1735/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0469 - categorical_accuracy: 0.9899
Epoch 1736/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.0731 - categorical_accuracy: 0.9821
Epoch 1737/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.0487 - categorical_accuracy: 0.9899
Epoch 1738/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.0739 - categorical_accuracy: 0.9821
Epoch 1739/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.0466 - categorical_accuracy: 0.9899
Epoch 1740/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0717 - categorical_accurac

3/3 [==============================] - 0s 47ms/step - loss: 0.0540 - categorical_accuracy: 0.9899
Epoch 1805/2000
3/3 [==============================] - 0s 52ms/step - loss: 0.0527 - categorical_accuracy: 0.9899
Epoch 1806/2000
3/3 [==============================] - 0s 47ms/step - loss: 0.0537 - categorical_accuracy: 0.9899
Epoch 1807/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.0818 - categorical_accuracy: 0.9821
Epoch 1808/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.0552 - categorical_accuracy: 0.9899
Epoch 1809/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0811 - categorical_accuracy: 0.9821
Epoch 1810/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.0527 - categorical_accuracy: 0.9899
Epoch 1811/2000
3/3 [==============================] - 0s 38ms/step - loss: 0.0386 - categorical_accuracy: 0.9938
Epoch 1812/2000
3/3 [==============================] - 0s 53ms/step - loss: 0.0805 - categorical_accurac

3/3 [==============================] - 0s 37ms/step - loss: 0.0747 - categorical_accuracy: 0.9821
Epoch 1877/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.0344 - categorical_accuracy: 0.9938
Epoch 1878/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.0746 - categorical_accuracy: 0.9821
Epoch 1879/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0486 - categorical_accuracy: 0.9899
Epoch 1880/2000
3/3 [==============================] - 0s 36ms/step - loss: 0.0478 - categorical_accuracy: 0.9899
Epoch 1881/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0356 - categorical_accuracy: 0.9938
Epoch 1882/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0750 - categorical_accuracy: 0.9821
Epoch 1883/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0486 - categorical_accuracy: 0.9899
Epoch 1884/2000
3/3 [==============================] - 0s 40ms/step - loss: 0.0736 - categorical_accurac

3/3 [==============================] - 0s 34ms/step - loss: 0.0455 - categorical_accuracy: 0.9899
Epoch 1949/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0728 - categorical_accuracy: 0.9821
Epoch 1950/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0724 - categorical_accuracy: 0.9821
Epoch 1951/2000
3/3 [==============================] - 0s 34ms/step - loss: 0.0471 - categorical_accuracy: 0.9899
Epoch 1952/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0476 - categorical_accuracy: 0.9899
Epoch 1953/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0728 - categorical_accuracy: 0.9821
Epoch 1954/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0720 - categorical_accuracy: 0.9821
Epoch 1955/2000
3/3 [==============================] - 0s 35ms/step - loss: 0.0459 - categorical_accuracy: 0.9899
Epoch 1956/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.0735 - categorical_accurac

In [51]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 64)            442112    
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
________________________________________________

# 8. Make Predictions

In [52]:
res = model.predict(X_test)

In [53]:
actions[np.argmax(res[4])]

'Obrigado'

In [54]:
actions[np.argmax(y_test[4])]

'Genial'

# 9. Save Weights

In [55]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [56]:
model.save('action.h5')

In [ ]:
del model

In [57]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [58]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [59]:
yhat = model.predict(X_test)

In [60]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [61]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[6, 0],
        [0, 3]],

       [[5, 2],
        [0, 2]],

       [[5, 0],
        [2, 2]]])

In [62]:
accuracy_score(ytrue, yhat)

0.7777777777777778

# 11. Test in Real Time

In [63]:
from scipy import stats

In [64]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [65]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.py

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.S

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.pyt

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python

...

Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Genial
<class 'mediapipe.python.solution

Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Obrigado
<

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ola
<class 'mediapipe.python

KeyboardInterrupt: 

In [66]:
cap.release()
cv2.destroyAllWindows()